In [11]:
import pickle as pickle
import os
import pandas as pd
import torch, wandb, random
import sklearn
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import StratifiedShuffleSplit
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    RobertaConfig,
    RobertaTokenizer,
    RobertaForSequenceClassification,
    BertTokenizer,
)
from load_data import *
from torch.utils.data import Subset
import gc
import argparse
from utils import *

# 코드

In [ ]:
path="./results/"
checkpoints = search(path)
checkpoints = checkpoints[0] + checkpoints

In [18]:
model = []
for checkpoint in checkpoints:
    model.append(AutoModelForSequenceClassification.from_pretrained(path + '/' + checkpoint))

In [20]:
# 모델의 state_dict 가중평균 구하기

# 1. 0번째 모델의 state_dict를 모두 0으로 만들기

for param_tensor in list(model[0].state_dict())[1:]:
    model[0].state_dict()[param_tensor] -= model[0].state_dict()[param_tensor]

In [21]:
# 2. 1번째~3번째 모델의 state_dict/3 더하기

for i in range(1,len(model)):
    for param_tensor in list(model[i].state_dict())[1:]:
        model[0].state_dict()[param_tensor] += (model[i].state_dict()[param_tensor]/3).float()

In [22]:
# 모델의 state_dict 출력
print("Model's state_dict:")
for param_tensor in model[0].state_dict():
    print(param_tensor, "\t", model[0].state_dict()[param_tensor])

Model's state_dict:
roberta.embeddings.position_ids 	 tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
          42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
          56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
          70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
          84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
          98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
         112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
         126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
         140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
         154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167

In [23]:
weighted_model = model[0]
weighted_model.save_pretrained(f"./best_model/")

---